In [1]:
# Import necessary components
from src.retrieval import FaissRetriever
from src.guardrail import Guardrails
import openai
import os

# Set up OpenAI API key (assuming it's already set in your environment)
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")

class OpenAIGPT:
    def __init__(self, retriever):
        """
        Initialize the OpenAIGPT class using the FAISS retriever.
        """
        self.retriever = retriever

    def retrieve_and_generate_response(self, user_query, k=3):
        """
        This method performs two main tasks:
        1. Retrieve relevant products using the FAISS retriever.
        2. Generate a response using GPT-4 based on the retrieved product information.
        
        :param user_query: The user's query (e.g., "Tell me about credit cards").
        :param k: The number of products to retrieve.
        :return: A GPT-4 generated response.
        """
        # Step 1: Retrieve relevant products from the knowledge base using FAISS
        retrieved_products = self.retriever.search(user_query, k)
        
        # Step 2: Format the retrieved products into a text prompt for GPT-4
        product_info = "\n".join([
            f"Product: {p['product_name']}\nDescription: {p['description']}\n" for p in retrieved_products
        ])
        
        # Create the final prompt for GPT-4
        prompt = f"""
                    The user asked: '{user_query}'. Based on the following product information, provide a helpful response in the following JSON format:
                    {{
                    "response": "Your generated response here"
                    }}
                    {product_info}
                    """

        # Step 3: Use OpenAI GPT-4 to generate a response based on the prompt
        completion = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            temperature=0.7
        )
        
        # Extract and return the text of the generated response
        return completion.choices[0].message.content

class OpenAIGPTWithGuardrails:
    def __init__(self, retriever, guardrails):
        """
        Initialize the OpenAIGPTWithGuardrails class using the FAISS retriever and Guardrails for validation.
        """
        self.retriever = retriever
        self.guardrails = guardrails

    def retrieve_and_generate_response(self, user_query, k=3):
        """
        Retrieve relevant products using FAISS and generate a response with GPT-4. Validate the response using Guardrails.
        
        :param user_query: The user's query (e.g., "Tell me about credit cards").
        :param k: The number of products to retrieve.
        :return: A GPT-4 generated and validated response.
        """
        # Step 1: Retrieve relevant products from the knowledge base using FAISS
        retrieved_products = self.retriever.search(user_query, k)
        
        # Step 2: Format the retrieved products into a text prompt for GPT-4
        product_info = "\n".join([
            f"Product: {p['product_name']}\nDescription: {p['description']}\n" for p in retrieved_products
        ])
        
        # Create the final prompt for GPT-4
        prompt = f"""
                    The user asked: '{user_query}'. Based on the following product information, provide a helpful response in the following JSON format:
                    {{
                    "response": "Your generated response here"
                    }}
                    {product_info}
                    """
        
        # Step 3: Use OpenAI GPT-4 to generate a response based on the prompt
        completion = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            temperature=0.7
        )
        
        # Step 4: Extract the generated response
        generated_response = completion.choices[0].message.content.strip()
        
        # Step 5: Validate the response using Guardrails
        validated_response = self.guardrails.validate_response(generated_response)
        
        return validated_response

# Instantiate the FaissRetriever and Guardrails
retriever = FaissRetriever("data/products.json")
guardrails = Guardrails()

# Instantiate the GPT model without and with Guardrails
gpt_without_guardrails = OpenAIGPT(retriever)
gpt_with_guardrails = OpenAIGPTWithGuardrails(retriever, guardrails)

# Test the system with an example query
user_query = "Tell me about the credit card."
response_without_guardrails = gpt_without_guardrails.retrieve_and_generate_response(user_query)
response_with_guardrails = gpt_with_guardrails.retrieve_and_generate_response(user_query)

print(response_without_guardrails)
print(response_with_guardrails)


c:\Users\dmont\anaconda3\envs\cba-chatbot-guardrails-env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\dmont\anaconda3\envs\cba-chatbot-guardrails-env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\dmont\anaconda3\envs\cba-chatbot-guardrails-env\Lib\site-packages\guardrails\validator_service\__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


{
"response": "The product you are asking about is a Credit Card. It is designed to be a versatile tool for everyday purchases, offering rewards points to users. These rewards can be used for various benefits, which differ from card to card. However, it's always advisable to use the credit card wisely to avoid any potential debt issues."
}
ValidationOutcome(
    call_id='2815933640736',
    raw_llm_output='{\n"response": "The product you asked about is a Credit Card. It is a versatile card that you can use for your everyday purchases. One of the benefits of using this card is that you can earn rewards points. These reward points can typically be used to redeem various gifts or services, giving you added value for your spending."\n}',
    validation_summaries=None,
    validated_output={
        'response': 'The product you asked about is a Credit Card. It is a versatile card that you can use for your everyday purchases. One of the benefits of using this card is that you can earn reward